In [4]:
from datetime import datetime
from energy_box_control.power_hub.control import initial_control_state, no_control
from energy_box_control.power_hub.network import PowerHub, PowerHubSchedules
from energy_box_control.schedules import ConstSchedule

%load_ext autoreload 
%autoreload 2



schedules = PowerHubSchedules(global_irradiance_schedule=ConstSchedule(800))#, cooling_demand_schedule = ConstSchedule(100/24/60/60))
power_hub = PowerHub.power_hub(schedules)

state = power_hub.simulate(
    power_hub.simple_initial_state(start_time=datetime.now()),
    no_control(power_hub),
)

control_state = initial_control_state()
power_hub_sensors = power_hub.sensors_from_state(state)



The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [6]:
import json
from energy_box_control.json import SensorEncoder


json.dumps(power_hub_sensors, cls=SensorEncoder)

'{"heat_pipes": {"flow": 0, "ambient_temperature": 20, "global_irradiance": 800, "input_temperature": 20, "output_temperature": 20}, "heat_pipes_valve": {"position": 0}, "hot_switch_valve": {"position": 0, "input_temperature": 20, "flow": 0}, "hot_reservoir": {"temperature": 20.0, "fill_flow": 0, "fill_output_temperature": 20.0}, "pcm": {"temperature": 20.0, "discharge_input_temperature": 20, "discharge_output_temperature": 20, "discharge_flow": 0}, "yazaki_hot_bypass_valve": {"position": 0}, "yazaki": {"hot_flow": 0, "hot_input_temperature": 20, "hot_output_temperature": 20}, "chiller": {}, "chiller_switch_valve": {"position": 0, "input_temperature": 20}, "cold_reservoir": {"temperature": 20.024968789013734, "fill_flow": 1, "fill_input_temperature": 40, "fill_output_temperature": 20.024968789013734, "exchange_flow": 0, "exchange_input_temperature": 20.0, "exchange_output_temperature": 20.024968789013734}, "waste_bypass_valve": {"position": 0, "input_temperature": 20}, "preheat_switch_

In [7]:
import json
from energy_box_control.json import encoder
import pandas as pd

pd.read_json(json.dumps(power_hub_sensors, cls=SensorEncoder))

/var/folders/w7/x3ddjmps2dvfqdrygwzccnd00000gn/T/ipykernel_27604/1495822923.py:5: FutureWarning: Passing literal json to 'read_json' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  pd.read_json(json.dumps(power_hub_sensors, cls=SensorEncoder))


,heat_pipes,heat_pipes_valve,hot_switch_valve,hot_reservoir,pcm,yazaki_hot_bypass_valve,yazaki,chiller,chiller_switch_valve,cold_reservoir,waste_bypass_valve,preheat_switch_valve,preheat_reservoir,waste_switch_valve,outboard_exchange,hot_mix
flow,0.0,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.666667,NaN
ambient_temperature,20.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
global_irradiance,800.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
input_temperature,20.0,NaN,20.0,NaN,NaN,NaN,NaN,NaN,20.0,NaN,20.0,20.0,NaN,20.0,20.000000,NaN
output_temperature,20.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,23.011956,20.0
position,NaN,0.0,0.0,NaN,NaN,0.0,NaN,NaN,0.0,NaN,0.0,1.0,NaN,0.0,NaN,NaN
temperature,NaN,NaN,NaN,20.0,20.0,NaN,NaN,NaN,NaN,20.024969,NaN,NaN,20.0,NaN,NaN,NaN
fill_flow,NaN,NaN,NaN,0.0,NaN,NaN,NaN,NaN,NaN,1.000000,NaN,NaN,NaN,NaN,NaN,NaN
fill_output_temperature,NaN,NaN,NaN,20.0,NaN,NaN,NaN,NaN,NaN,20.024969,NaN,NaN,NaN,NaN,NaN,NaN
discharge_input_temperature,NaN,NaN,NaN,NaN,20.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
